In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from SALib.sample.morris.strategy import Strategy
from SALib.sample.morris import generate_trajectory, generate_trajectory_2, _sample_oat, \
generate_p_star, generate_x_star, compute_b_star

In [2]:
sample = _sample_oat({'num_vars': 10}, 5)

In [3]:
sample.shape

(55, 10)

In [4]:
sample[:11, 4]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.33333333, 0.33333333,
       0.33333333])

## `sample` matrix

The way that this array is constructed is... unintuituve. Maybe it is like this for backwards compatibility. `sample` is a two-dimensional array which includes three dimensions: trajectory, parameter, and parameter value. Personally, I would prefer this to be a three dimensional array (order doesn't really matter).

Note: My (older, unmaintained) [py-elem-effects](https://bitbucket.org/cmutel/py-elem-effects/) library doesn't store the coordinates of every single point at each point on the trajectory, but rather the starting and ending values for each parameter, and the order the parameters change in. This means you reduce memory usage substantially.

The rows are parameter values for the first trajectory, follow by for the second trajectory, etc. Columns are the parameters.

## Trajectory generation performance

The function `_sample_oat` includes this line:

    sample = np.array([generate_trajectory(group_membership, num_levels)
                       for n in range(N)])

These types of loops should almost always be avoided, as there are numpy operators that are much faster. Let's see if we can make trajectory generation faster.

In [5]:
m = np.asmatrix(np.identity(1000, dtype=int))

In [6]:
m.shape

(1000, 1000)

In [ ]:
%timeit generate_trajectory(m, 4)

In [ ]:
type(generate_trajectory(m, 4))

In [ ]:
%timeit generate_trajectory_2(m, 4)

In [ ]:
type(generate_trajectory_2(m, 4))

In [7]:
import numpy.random as rd

In [17]:
rd.seed(5)
num_levels = 4

delta = float(num_levels) / (2 * (num_levels - 1))
    
group_membership = m

# Infer number of groups `g` and number of params `k` from
# `group_membership` matrix
num_params, num_groups = group_membership.shape

# Matrix B - size (g + 1) * g -  lower triangular matrix
B = np.tril(np.ones([num_groups + 1, num_groups], dtype=int), -1)

P_star = generate_p_star(num_groups)

# Matrix J - a (g+1)-by-num_params matrix of ones
J = np.ones((num_groups + 1, num_params))

# Matrix D* - num_params-by-num_params matrix which decribes whether
# factors move up or down
D_star = np.diag(rd.choice([-1,1],size=num_params))

x_star = generate_x_star(num_params, num_levels)

# Matrix B* - size (num_groups + 1) * num_params
B_star1 = compute_b_star(J, x_star, delta, B,
                        group_membership, P_star, D_star)

B_star2 = np.multiply(( x_star + delta/2. * ( np.dot((2*B - 1),D_star) + 1 ) ), P_star)



ValueError: operands could not be broadcast together with shapes (1001,1000) (1000,1000) 

In [ ]:
B_star1[0]

In [ ]:
B_star2[0]

In [18]:
np.multiply?

In [ ]:
G = m

element_a = np.ones((1,num_params)) * x_star
element_b = np.matmul(G, P_star).T
element_c = np.matmul(2 * B, element_b)
element_d = np.matmul((element_c - J), D_star)

b_star = element_a + (delta / 2) * (element_d + J)

In [ ]:
np.all(element_a == x_star)

In [ ]:
x_star